<a href="https://colab.research.google.com/github/meinternational/HMF/blob/main/HMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==1.15
!pip install nilearn
!pip install nibabel

     |████████████████████████████████| 411.5MB 41kB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 512kB 51.5MB/s 
     |████████████████████████████████| 3.8MB 50.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e67ed86e5737990aaa75cae571cb9a55bd2c05b30bb5d6dcd8ebc32d1af4ad1e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

In [ ]:
!wget https://github.com/meinternational/HMF/releases/download/v0.1/msc_input_motor_session01.npy
!wget https://github.com/meinternational/HMF/releases/download/v0.1/msc_settings_motor_session01.npy

In [ ]:
import tensorflow as tf
from tensorflow.contrib.opt import ScipyOptimizerInterface
import numpy as np
import nibabel as nib
from numpy.random import seed

tf_f = dict()
tf_f['tensorflow.nn.relu'] = tf.nn.relu
tf_f['tensorflow.nn.sigmoid'] = tf.nn.sigmoid
tf_f['tensorflow.abs'] = tf.abs
tf_f['tensorflow.identity'] = tf.identity
tf_f['tensorflow.tanh'] = tf.tanh
tf_f['tensorflow.nn.softplus'] = tf.nn.softplus

class Base(object):

    def __init__(self, settings):
        self.settings = settings
        self.placeholder = self.build_placeholder()
        self.model = self.build_model()
        if 'lbfgsb' in self.settings and self.settings['lbfgsb']:
            self.optimizer = ScipyOptimizerInterface(self.model['cost'], method='L-BFGS-B', options={'maxiter': self.settings['max_iter'], 'disp': True})
        else:
            optimizer = tf.train.AdamOptimizer()
            self.optimizer = optimizer.minimize(self.model['cost'])
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
        self.sess.run(init)

    def g(self, h, cost='l1'):
        if cost=='logcosh':
            return tf.reduce_sum( 0.5 * tf.log(tf.cosh(2*h)))

        if cost=='l1':
            return tf.reduce_sum(tf.abs(h))

        if cost=='l2':
            return tf.reduce_sum(tf.pow(h,2))

        if cost=='l1_approx':
            return tf.reduce_sum(tf.sqrt(tf.constant(10e-8) + tf.pow(h,2)))

        if cost=='exp':
            return tf.reduce_sum(-tf.exp(-tf.pow(h,2)/2.0))

    def kl_divergence(self,x, y): # Kullback-Leibler divergence between two exponential distributions
        return tf.log( x ) - tf.log( y ) + ( y / x ) - 1.0

    def filter2toeplitz(self, conv_filter): # converts filter to Toeplitz matrix
        len_h = conv_filter.get_shape().as_list()[0]
        toplitz = list()
        for t in range(self.settings['n_input']):
            if t==0:
                toplitz.append(tf.concat([conv_filter, tf.zeros(self.settings['n_input']-t-len_h)],0))

            elif t>0 and (t+len_h<self.settings['n_input']):
                toplitz.append(tf.concat([tf.zeros(t), conv_filter, tf.zeros(self.settings['n_input']-t-len_h)],0))

            else:
                toplitz.append(tf.concat([tf.zeros(t), conv_filter[0:self.settings['n_input']-t]],0))

        H = tf.transpose(tf.reshape(tf.concat(toplitz,0),(self.settings['n_input'],self.settings['n_input'])))
        return H

    def toeplitz2filter(self, H): # converts Toeplitz matrix to filter
        hw = self.settings['filter_length']/2
        hrft = list()
        for i,c in enumerate(range(hw,self.settings['n_input']-(hw+1))):
            hrft.append( H[c-hw:c+hw,c] )

        return tf.add_n(hrft)/(float(len(hrft)))

    def build_model(self): # overwrite this function
        pass

    def build_placeholder(self):
        placeholder = dict()
        placeholder['x'] = tf.placeholder(tf.float32)
        placeholder['GM'] = tf.placeholder(tf.float32)
        placeholder['MASK'] = tf.placeholder(tf.float32)
        if 'filter_length' in self.settings:
            placeholder['t'] = tf.placeholder(tf.float32, [self.settings['filter_length']])

        if 'H_init' in self.settings:
            placeholder['H_mask'] = tf.placeholder(tf.float32)

        return placeholder

    def fit(self, input_dict):
        feed_dict = dict()
        for key in list(self.placeholder.keys()):
            feed_dict[self.placeholder[key]] = input_dict[key]

        self.optimizer.minimize(self.sess, feed_dict=feed_dict)
        return self

    def get_params(self, input_dict):
        feed_dict = dict()
        for key in list(self.placeholder.keys()):
            feed_dict[self.placeholder[key]] = input_dict[key]

        out = self.sess.run(self.model, feed_dict=feed_dict)
        self.sess.close()
        tf.reset_default_graph()
        return out

class CanonicalHRFMatrixFactorizationFast(Base):

    def __init__(self, settings):
        super(CanonicalHRFMatrixFactorizationFast, self).__init__(settings)

    def build_model(self):
        model = dict()
        ######################### VARS ###################
        model['b1'] = tf.Variable(tf.zeros([self.settings['n_hidden'], 1], dtype=tf.float32), trainable=self.settings['train_b1'])
        model['neural'] = tf.nn.l2_normalize((tf_f[self.settings['f(neural)']](tf.get_variable("neural", dtype=tf.float32, shape=[
                                                self.settings['n_hidden'], self.settings['n_input']], initializer=tf.contrib.layers.xavier_initializer()))), 1) 

        model['hrf'] = tf.Variable(self.settings['hrfi'], dtype=tf.float32, trainable=self.settings['train_hrf'])
        model['b2'] = tf.Variable(tf.zeros([self.settings['n_input'], 1], dtype=tf.float32), trainable=self.settings['train_b2'])
        ######################## MODEL ###################
        H = self.filter2toeplitz(model['hrf'])
        model['bold'] = tf.transpose(tf.matmul(H, tf.transpose(model['neural'])))
        model['h'] = self.placeholder['MASK'] * tf_f[self.settings['f(WX)']](tf.matmul(model['bold'], self.placeholder['x']) + model['b1'])
        ######################## COST ###################
        model['l2'] = tf.nn.l2_loss(self.placeholder['GM'] * tf.subtract((tf.matmul(tf.transpose(model['bold']), model['h']) + model['b2']), self.placeholder['x'])) / (self.settings['n_feature'])
        cost = list()
        cost.append(model['l2'])

        if self.settings['lambda1'] > 0.0: # sparsity
            model['rho_hat'] = tf.reduce_sum(model['h'], axis=1, keepdims=True) / self.settings['n_feature']
            model['kl_space'] = tf.reduce_sum(self.kl_divergence(1.0/model['rho_hat'], 1.0/self.settings['mu']))
            model['lambda1_c'] = self.settings['lambda1'] * model['kl_space']
            cost.append(model['lambda1_c'])
        
        if self.settings['lambda2'] > 0.0: # total variation neural
            model['lambda2_c'] = self.settings['lambda2'] * tf.reduce_sum(self.g(
                model['neural'][:, 1:] - model['neural'][:, :-1], self.settings['g'])) / self.settings['n_input']
            cost.append(model['lambda2_c'])

        if self.settings['lambda2a'] > 0.0: # l1 approx neural
            model['lambda2a_c'] = self.settings['lambda2a'] * tf.reduce_sum(self.g(model['neural'], 'l1')) / self.settings['n_input']
            cost.append(model['lambda2a_c'])

        if self.settings['lambda2b'] > 0.0: # l2 neural
            model['lambda2b_c'] = self.settings['lambda2b'] * tf.reduce_sum(self.g(model['neural'], 'l2')) / self.settings['n_input']
            cost.append(model['lambda2b_c'])

        if self.settings['lambda3'] > 0.0: # total variation spatial
            MASK3D = tf.reshape(self.placeholder['MASK'], self.settings['dims']+(1,))
            H3D = tf.reshape(tf.transpose(model['h']), self.settings['dims']+(self.settings['n_hidden'],))

            model['lambda3_c'] = self.settings['lambda3'] * tf.reduce_sum(self.g(MASK3D[1:, :, :, :]*(H3D[1:, :, :, :] - H3D[:-1, :, :, :]), self.settings['g'])) / self.settings['n_feature'] \
                                + self.settings['lambda3'] * tf.reduce_sum(self.g(MASK3D[:, 1:, :, :]*(H3D[:, 1:, :, :] - H3D[:, :-1, :, :]), self.settings['g'])) / self.settings['n_feature'] \
                                + self.settings['lambda3'] * tf.reduce_sum(self.g(MASK3D[:, :, 1:, :]*(H3D[:, :, 1:, :] - H3D[:, :, :-1, :]), self.settings['g'])) / self.settings['n_feature']

            cost.append(model['lambda3_c']) 

        model['cost'] = tf.add_n(cost)

        return model

In [ ]:
settings = np.load('msc_settings_motor_session01.npy', allow_pickle=True).tolist()
input = np.load('msc_input_motor_session01.npy', allow_pickle=True).tolist()
input['x'] -=input['x'].mean(axis=0)
input['x'] /=input['x'].std(axis=0)

In [ ]:
settings['lambda2a'] = 1.0
settings['lambda2b'] = 20.0

In [ ]:
tf.reset_default_graph()
seed(42)
tf.random.set_random_seed(seed = 42)
hmf = CanonicalHRFMatrixFactorizationFast(settings)
hmf.fit(input)
out = hmf.get_params(input)

In [ ]:
print(out.keys())
print(input.keys())
print(settings.keys())
print(out['h'].shape)
print(np.min(input['GM']),np.max(input['GM']))
dims = settings['dims']
affine = np.array([[   4.,   -0.,   -0.,  -94.],
                [  -0.,    4.,   -0., -130.],
                [   0.,    0.,    4.,  -76.],
                [   0.,    0.,    0.,    1.]])
spatialmodes = nib.Nifti1Image(out['h'].T.reshape(dims[0],dims[1],dims[2],-1), affine=affine)
temporalmodes = out['neural']

In [ ]:
from nilearn.plotting import plot_glass_brain
from nilearn.image import index_img
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(30,120))
comps = [i for i in range(temporalmodes.shape[0])]#[1,2,9,11,13,35,36]
grid = plt.GridSpec(len(comps), 6, wspace=0.4, hspace=0.3)
for idx,comp in enumerate(comps):
  spatial_window = fig.add_subplot(grid[idx, 0])
  temporal_window = fig.add_subplot(grid[idx, 1:])
  plot_glass_brain(index_img(spatialmodes,comp), axes=spatial_window, title=idx)
  temporal_window.plot(temporalmodes[comp,:4*104])

In [ ]:
from nilearn.plotting import plot_glass_brain
from nilearn.image import index_img
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30,20))
comps = [1,13,17,2,22,36,20]
grid = plt.GridSpec(len(comps), 6, wspace=0.4, hspace=0.3)
for idx,comp in enumerate(comps):
  spatial_window = fig.add_subplot(grid[idx, 0])
  temporal_window = fig.add_subplot(grid[idx, 1:])
  plot_glass_brain(index_img(spatialmodes,comp), axes=spatial_window, title=idx)
  temporal_window.plot(temporalmodes[comp,:4*104])

fig.savefig('networks.png')